# About
Use similarity score of input text and category description to predict input text's category.

Initial phase:
1. Create text description for each category.
2. Get the embeddings of each cateogry description.
Inference phase:
1. Get embedding of the input text.
2. Calculate similarity of input text's embedding against category description embeddings.
3. Based upon similarity score, predict category of the input text.

Fine tuning:
1. Improvise category description.
2. User better model for getting embeddings.

# Data

In [5]:
# categories = {
#     "affirm": "Text that expresses agreement, confirmation, or a positive response. Examples: 'Yes, I agree,' 'That’s correct,' or 'Sure, I’ll do it.'",
#     "deny": "Text that expresses disagreement, refusal, or a negative response. Examples: 'No, that’s not right,' 'I disagree,' or 'I can’t do it.'",
#     "not_sure": "Text that expresses uncertainty, doubt, or hesitation. Examples: 'I’m not sure,' 'Maybe,' or 'I need more information.'"
# }
categories = {
    "affirm": "agreement confirmation positive Yes, I agree That’s correct or Sure I’ll do it.",
    "deny": "disagreement refusal negative No, that’s not right I disagree I can’t do it.",
    "not_sure": "uncertainty doubt hesitation I’m not sure Maybe I need more information."
}

In [6]:
test_samples = [
    # Affirmative Examples
    {"text": "Yes, that makes perfect sense.", "category": "affirm"},
    {"text": "I completely agree with you.", "category": "affirm"},
    {"text": "Absolutely, that’s correct.", "category": "affirm"},
    {"text": "Sure thing, I’ll handle it.", "category": "affirm"},

    # Negative Examples
    {"text": "No, I don’t think so.", "category": "deny"},
    {"text": "That’s not what I meant at all.", "category": "deny"},
    {"text": "I can’t agree with this.", "category": "deny"},
    {"text": "No way, that’s incorrect.", "category": "deny"},

    # Uncertainty Examples
    {"text": "I’m not sure if this is right.", "category": "not_sure"},
    {"text": "Maybe, but I need more information.", "category": "not_sure"},
    {"text": "It could be true, but I’m not confident.", "category": "not_sure"},
    {"text": "I don’t know for certain.", "category": "not_sure"},

    # Ambiguous/Edge Cases
    {"text": "Possibly, but I’ll have to think about it.", "category": "not_sure"},  # Uncertain tone
    {"text": "No, wait... actually, yes.", "category": "affirm"},  # Affirmative conclusion
    {"text": "I don’t think I agree with that, but I’m not certain.", "category": "not_sure"},  # Mixed tone
    {"text": "I’m sorry, I can’t do that.", "category": "deny"},  # Negative with apology
    {"text": "Yes, but I’m still a bit unsure.", "category": "affirm"},  # Affirmative with slight doubt
]

# Initial Phase
Getting embeddings of category descriptions.

## Using ollama
ollama exposes `/api/embeddings` api endpoint which we can use to get embeddings.

In [7]:
import ollama  # pip install ollama

# MODEL = "nomic-embed-text"
MODEL = "mxbai-embed-large"
HOST = "http:/localhost:11434"

In [8]:
category_embeddings = {
    category: ollama.embed(model=MODEL, input=description)["embeddings"][0]
    for category, description in categories.items()
}
category_embeddings

{'affirm': [0.00016016015,
  0.0018485776,
  -0.043112993,
  0.045043778,
  -0.04202081,
  0.027840164,
  0.021947172,
  0.026527898,
  0.06129573,
  0.054574005,
  -0.0076282574,
  0.026960807,
  0.017540218,
  -0.012052191,
  -0.022025771,
  0.003952206,
  -0.046570282,
  -0.015611735,
  -0.02454579,
  0.027886236,
  0.0023632043,
  -0.0012377824,
  -0.046685234,
  -0.015032398,
  -0.0058169225,
  0.05635849,
  -0.03166776,
  -0.024691626,
  0.06729083,
  0.045543734,
  -0.040106487,
  0.065838285,
  0.03653094,
  -0.010655208,
  -0.031598054,
  -0.052991364,
  0.055408273,
  -0.005580624,
  -0.01695822,
  -0.040271938,
  -0.020454893,
  0.021116493,
  0.009470924,
  -0.024472708,
  -0.041430023,
  -0.06808195,
  -0.04552576,
  -0.055052057,
  0.030613879,
  -0.08669417,
  0.007987908,
  0.033284135,
  0.055396702,
  -0.03312707,
  0.03464766,
  0.020730622,
  -0.027052807,
  0.0011477004,
  -0.038004003,
  0.038563937,
  -0.011173461,
  -0.028681966,
  0.016291438,
  -0.060145363,
 

# Inference
Classify the category of input text.

In [9]:
from sklearn.metrics.pairwise import cosine_similarity


def cal_similarity(v1, v2):
    return cosine_similarity([v1], [v2])


def classify(input_text: str) -> str:
    """
    Args:
        input (str): The input text that is to be classified.
    Returns:
        str: The predicted class of input text.
    """
    input_text_embedding = ollama.embed(model=MODEL, input=input_text)["embeddings"][0]
    similarities = {
        category: cal_similarity(input_text_embedding, category_embedding)
        for category, category_embedding in category_embeddings.items()
    }
    return max(similarities, key=similarities.get)


classify("Yea sure!")

'affirm'

# Evaluation

In [10]:
for sample in test_samples:
    predicted = classify(sample["text"])
    print(f'{predicted} : {sample["category"]} : {sample["text"]}')

affirm : affirm : Yes, that makes perfect sense.
affirm : affirm : I completely agree with you.
affirm : affirm : Absolutely, that’s correct.
affirm : affirm : Sure thing, I’ll handle it.
deny : deny : No, I don’t think so.
deny : deny : That’s not what I meant at all.
deny : deny : I can’t agree with this.
deny : deny : No way, that’s incorrect.
not_sure : not_sure : I’m not sure if this is right.
not_sure : not_sure : Maybe, but I need more information.
not_sure : not_sure : It could be true, but I’m not confident.
not_sure : not_sure : I don’t know for certain.
not_sure : not_sure : Possibly, but I’ll have to think about it.
affirm : affirm : No, wait... actually, yes.
not_sure : not_sure : I don’t think I agree with that, but I’m not certain.
deny : deny : I’m sorry, I can’t do that.
not_sure : affirm : Yes, but I’m still a bit unsure.
